In [5]:
import pickle
import bisect

import numpy as np
import tqdm

import flowmap.formats.ugrid
import flowmap.dem


In [6]:
with open('/Users/baart_f/data/rijnland/vanGovert/Leerdam/leerdamwest_map_tables.pckl', 'rb') as f:
    table = pickle.load(f)
   

In [7]:
dem = flowmap.dem.read_dem('/Users/baart_f/data/rijnland/vanGovert/Leerdam/aw_refi_def_asc.tiff')
grid = flowmap.formats.ugrid.UGrid('/Users/baart_f/data/rijnland/vanGovert/Leerdam/leerdamwest_map.nc')

In [8]:
table.head()

,bins,cum_volume_table,extent,face,n,slice,volume_table
id,,,,,,,
0,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433491.214474, 433493...","[[134591.854, 433493.214474], [134589.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2381, 2385, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433497.214474, 433499...","[[134591.854, 433497.214474], [134591.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2369, 2373, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433495.214474, 433497...","[[134591.854, 433497.214474], [134589.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2373, 2377, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433493.214474, 433495...","[[134591.854, 433493.214474], [134591.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2377, 2381, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433489.214474, 433491...","[[134591.854, 433489.214474], [134591.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2385, 2389, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [12]:
def subgrid_waterlevel(row, dem):
    """get the subgrid waterdepth image"""
    # tables is a dataframe
    bins = row["bins"]

    # if we don't have any volume table
    if bins is None:
        return None
    volume_table = row["volume_table"]
    cum_volume_table = row["cum_volume_table"]

    dem_i = dem['band'][row['slice']]

    # this part is once volume is known
    vol_i = row['vol1']
    
    fill_idx = bisect.bisect(cum_volume_table, vol_i)
    remaining_volume = vol_i - cum_volume_table[fill_idx - 1]
    pixel_area = row['pixel_area']
    face_area = np.prod(dem_i.shape) * pixel_area

    if fill_idx >= len(cum_volume_table) - 1:
        remaining = (vol_i - cum_volume_table[-1]) / face_area
        target_level = bins[-1] + remaining
    else:
        remaining_volume_fraction = remaining_volume / volume_table[fill_idx]
        target_level = bins[fill_idx] + remaining_volume_fraction * (bins[fill_idx + 1] - bins[fill_idx])

    # first cell that is not completely filled
    return target_level


In [13]:
for key, val in grid.waterlevel(-1).items():
    table[key] = val

table['pixel_area'] = 0.5 * 0.5
table.head()

,bins,cum_volume_table,extent,face,n,slice,volume_table,s1,vol1,waterdepth,pixel_area
id,,,,,,,,,,,
0,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433491.214474, 433493...","[[134591.854, 433493.214474], [134589.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2381, 2385, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.118593,0.359522,0.089875,0.25
1,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433497.214474, 433499...","[[134591.854, 433497.214474], [134591.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2369, 2373, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12.113614,0.009255,0.002320,0.25
2,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433495.214474, 433497...","[[134591.854, 433497.214474], [134589.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2373, 2377, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.118208,0.564350,0.141065,0.25
3,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433493.214474, 433495...","[[134591.854, 433493.214474], [134591.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2377, 2381, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.118256,0.536556,0.134137,0.25
4,"[1.5, 1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[134589.854, 134591.854, 433489.214474, 433491...","[[134591.854, 433489.214474], [134591.854, 433...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0,...","(slice(2385, 2389, None), slice(4360, 4364, No...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.118652,0.441780,0.110459,0.25


In [14]:
# register listener to pandas
tqdm.tqdm_notebook(desc="panda is out for lunch!").pandas()
table['subgrid_waterlevel'] = table.progress_apply(subgrid_waterlevel, axis=1, dem=dem)

HBox(children=(IntProgress(value=1, bar_style='info', description='panda is out for lunch!', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=546506), HTML(value='')))

In [15]:
import geojson

    

In [16]:
def row2feature(row, crs):
    
    coordinates = row['face'].mean(axis=0)
    feature = geojson.Feature(
        geometry=geojson.Point(
            coordinates=tuple(coordinates),
            crs=crs
        ),
        id=int(row.name),
        properties={
            "s1": float(row.s1),
            "subgrid_waterlevel": float(row.subgrid_waterlevel),
            "vol1": float(row.vol1),
            "waterdepth": float(row.waterdepth)
        },
        crs=crs
    )
    return feature


In [17]:
crs = geojson.crs.Named(
    properties={
        "name": "urn:ogc:def:crs:EPSG::28992"
    }
)
features = list(
    table.progress_apply(row2feature, axis=1, crs=crs)
)


HBox(children=(IntProgress(value=0, max=546506), HTML(value='')))

In [18]:
collection = geojson.FeatureCollection(features=features, crs=crs)
with open('test.json', 'w') as f:
    geojson.dump(collection, f)